In [ ]:
# from datasets import load_dataset

# dataset = load_dataset(
#     "slegroux/tiny-imagenet-200-clean",
#     cache_dir="/kaggle/working/hf_cache"
# )

# print(dataset)

In [ ]:
# import os

# out_root = "/kaggle/working/tiny-imagenet-200"
# os.makedirs(out_root, exist_ok=True)

# for split in dataset:
#     for i, sample in enumerate(dataset[split]):
#         label = sample.get("label", "unknown")

#         if split == "test":
#             out_dir = os.path.join(out_root, split, "images")
#         else:
#             out_dir = os.path.join(out_root, split, str(label))

#         os.makedirs(out_dir, exist_ok=True)
#         sample["image"].save(os.path.join(out_dir, f"{i}.png"))


In [ ]:
# working_dir = "/kaggle/working/tiny-imagenet-200"

In [ ]:
import torch
from torchvision import transforms
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import transforms
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader, random_split
import pandas as pd
import os
from sklearn.model_selection import train_test_split
from PIL import Image
from datasets import load_dataset

In [ ]:
# class AlexNet(nn.Module):
#     def __init__(self, num_classes, task='classification'):
#         super().__init__()
#         # Only define structure, not computation
#         self.conv1 = nn.Conv2d(in_channels=3,
#                                out_channels=96,
#                                kernel_size=(11,11),
#                                stride=4
#                               )
#         self.conv2 = nn.Conv2d(in_channels=96,
#                                out_channels=256, 
#                                kernel_size=(5,5),
#                                padding=2
#                               )
#         self.conv3 = nn.Conv2d(in_channels=256,
#                                out_channels=384,
#                                kernel_size=(3,3),
#                                padding=1
#                               )
#         self.conv4 = nn.Conv2d(in_channels=384,
#                                out_channels=384,
#                                kernel_size=(3,3),
#                                padding=1
#                               )
#         self.conv5 = nn.Conv2d(in_channels=384,
#                                out_channels=256,
#                                kernel_size=(3,3),
#                                padding=1
#                               )
#         self.lrn1 = nn.LocalResponseNorm(size=5, k = 2, alpha=1e-4, beta=0.75)
#         self.lrn2 = nn.LocalResponseNorm(size=5, k = 2, alpha=1e-4, beta=0.75)

#         self.max_pool = nn.MaxPool2d(kernel_size=3, stride=2)
#         self.relu = nn.ReLU()
#         self.dropout = nn.Dropout()
        
#         self.fc1 = nn.Linear(6*6*256, 4096)
#         self.fc2 = nn.Linear(4096, 4096)
#         self.fc3 = nn.Linear(4096, num_classes)

#     def forward(self, x):
#         x = self.max_pool(self.lrn1(self.relu(self.conv1(x))))
#         x = self.max_pool(self.lrn2(self.relu(self.conv2(x))))
#         x = self.relu(self.conv3(x))
#         x = self.relu(self.conv4(x))
#         x = self.max_pool(self.relu(self.conv5(x)))
#         x = self.dropout(self.relu(self.fc1(x.flatten(start_dim=1))))
#         x = self.dropout(self.relu(self.fc2(x)))
#         x = self.fc3(x)
#         return x

In [ ]:
class AlexNet(nn.Module):
    def __init__(self, num_classes=1000):
        super().__init__()

        # Conv1 (GPU 0)
        self.conv1 = nn.Conv2d(3, 96, 11, stride=4).to("cuda:0")
        self.lrn1 = nn.LocalResponseNorm(5, k=2, alpha=1e-4, beta=0.75).to("cuda:0")

        # Conv2 (split)
        self.conv2_gpu0 = nn.Conv2d(48, 128, 5, padding=2).to("cuda:0")
        self.conv2_gpu1 = nn.Conv2d(48, 128, 5, padding=2).to("cuda:1")
        
        self.lrn2_gpu0 = nn.LocalResponseNorm(5, k=2, alpha=1e-4, beta=0.75).to("cuda:0")
        self.lrn2_gpu1 = nn.LocalResponseNorm(5, k=2, alpha=1e-4, beta=0.75).to("cuda:1")

        # Conv3 (GPU 0)
        self.conv3 = nn.Conv2d(256, 384, 3, padding=1).to("cuda:0")

        # Conv4 (split)
        self.conv4_gpu0 = nn. Conv2d(192, 192, 3, padding=1).to("cuda:0")
        self.conv4_gpu1 = nn.Conv2d(192, 192, 3, padding=1).to("cuda:1")

        # Conv5 (split)
        self.conv5_gpu0 = nn.Conv2d(192, 128, 3, padding=1).to("cuda:0")
        self.conv5_gpu1 = nn.Conv2d(192, 128, 3, padding=1).to("cuda:1")

        # FC layers
        self.fc1_gpu0 = nn.Linear(128 * 6 * 6, 2048).to("cuda:0")
        self.fc1_gpu1 = nn.Linear(128 * 6 * 6, 2048).to("cuda:1")  # Fixed spacing
        
        self.fc2_gpu0 = nn.Linear(2048, 2048).to("cuda:0")
        self.fc2_gpu1 = nn.Linear(2048, 2048).to("cuda:1")
        
        self. fc3 = nn.Linear(4096, num_classes).to("cuda:0")
        
        self.dropout = 0.5

    def forward(self, x):
        # Input starts on GPU 0
        x = x.to("cuda:0")

        # -------- Conv1 --------
        x = F. relu(self.conv1(x))
        x = self.lrn1(x)
        x = F.max_pool2d(x, kernel_size=3, stride=2)

        # Split channels (96 -> 48 + 48)
        x0, x1 = torch.chunk(x, 2, dim=1)
        x1 = x1.to("cuda:1")

        # -------- Conv2 (isolated) --------
        x0 = F.relu(self.conv2_gpu0(x0))
        x1 = F.relu(self.conv2_gpu1(x1))

        x0 = self.lrn2_gpu0(x0)
        x1 = self.lrn2_gpu1(x1)

        x0 = F.max_pool2d(x0, kernel_size=3, stride=2)
        x1 = F. max_pool2d(x1, kernel_size=3, stride=2)  # Fixed spacing

        # -------- Conv3 (merge) --------
        x1 = x1.to("cuda:0")
        x = torch.cat([x0, x1], dim=1)
        x = F.relu(self.conv3(x))

        # Split again (384 -> 192 + 192)
        x0, x1 = torch.chunk(x, 2, dim=1)
        x1 = x1.to("cuda:1")

        # -------- Conv4 --------
        x0 = F.relu(self.conv4_gpu0(x0))
        x1 = F.relu(self.conv4_gpu1(x1))

        # -------- Conv5 --------
        x0 = F.relu(self.conv5_gpu0(x0))
        x1 = F. relu(self. conv5_gpu1(x1))  # Fixed spacing

        x0 = F.max_pool2d(x0, kernel_size=3, stride=2)
        x1 = F.max_pool2d(x1, kernel_size=3, stride=2)

        # -------- FC layers --------
        x0 = x0.flatten(1)
        x1 = x1.flatten(1)

        # FC1
        z0 = F. dropout(F.relu(self. fc1_gpu0(x0)), p=self.dropout, training=self.training)
        z1 = F.dropout(F.relu(self.fc1_gpu1(x1)), p=self.dropout, training=self. training)

        # FC2
        z0 = F.dropout(F.relu(self.fc2_gpu0(z0)), p=self.dropout, training=self. training)  # Fixed spacing
        z1 = F.dropout(F.relu(self.fc2_gpu1(z1)), p=self.dropout, training=self.training)

        # Final merge
        z1 = z1.to("cuda:0")
        z = torch.cat([z0, z1], dim=1)

        return self.fc3(z)

In [ ]:
model = AlexNet(num_classes=200)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model.to(device)

In [ ]:
IMAGENET_EIGVAL = torch.tensor([0.2175, 0.0188, 0.0045])
IMAGENET_EIGVEC = torch.tensor([
    [-0.5675,  0.7192,  0.4009],
    [-0.5808, -0.0045, -0.8140],
    [-0.5836, -0.6948,  0.4203],
])

In [ ]:
class AlexNetPCAJitter(torch.nn.Module):
    def __init__(self, eigval, eigvec, alpha_std=0.1):
        super().__init__()
        self.eigval = eigval
        self.eigvec = eigvec
        self.alpha_std = alpha_std

    def forward(self, img):
        # img: Tensor (3, H, W) in [0,1]
        alpha = torch.randn(3) * self.alpha_std
        rgb_shift = (self.eigvec @ (alpha * self.eigval)).view(3, 1, 1)
        return img + rgb_shift

In [ ]:
import os
os.environ["HF_HUB_READ_TIMEOUT"] = "60"
os.environ["HF_HUB_CONNECT_TIMEOUT"] = "60"

In [ ]:
train_dataset = load_dataset('slegroux/tiny-imagenet-200-clean', split='train')                
valid_dataset = load_dataset('slegroux/tiny-imagenet-200-clean', split='validation')
test_dataset = load_dataset('slegroux/tiny-imagenet-200-clean', split='test')

In [ ]:
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.RandomCrop(227),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    AlexNetPCAJitter(IMAGENET_EIGVAL, IMAGENET_EIGVEC),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])
test_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(227),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])


In [ ]:
def preprocess(example, transform):
    example["image"] = [transform(img) for img in example["image"]]
    return example

train_dataset = train_dataset.with_transform(lambda x: preprocess(x, transform))
valid_dataset = valid_dataset.with_transform(lambda x: preprocess(x, test_transform))
test_dataset = test_dataset.with_transform(lambda x: preprocess(x, test_transform))

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=4)
val_loader   = DataLoader(valid_dataset, batch_size=128, shuffle=False, num_workers=4)
test_loader   = DataLoader(test_dataset, batch_size=128, shuffle=False, num_workers=4)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(),lr=0.01, momentum=0.9, weight_decay=0.5e-4)
# num_gpus = torch.cuda.device_count()
# base_lr = 0.02
# lr = base_lr * num_gpus   # Scale learning rate for DataParallel
# optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9, weight_decay=5e-5)

In [ ]:
def train_one_epoch(model, dataloader, optimizer, criterion, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for batch in dataloader:
        images = batch['image'].to(device)
        label = batch['label'].to(device)

        # Forward
        output = model(images)
        loss = criterion(output, label)

        # Backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Statistics
        running_loss += loss.item() * images.size(0)
        _, preds = output.max(1)
        correct += preds.eq(label).sum().item()
        total += label.size(0)
    epoch_loss = running_loss / total
    epoch_acc = correct / total
    return epoch_loss, epoch_acc

@torch.no_grad()
def validate(model, dataloader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0

    for batch in dataloader:
        images = batch['image'].to(device)
        label = batch['label'].to(device)

        # Forward
        output = model(images)
        loss = criterion(output, label)
        running_loss += loss.item() * images.size(0)
        _, preds = output.max(1)
        correct += preds.eq(label).sum().item()
        total += label.size(0)
    epoch_loss = running_loss / total
    epoch_acc = correct / total
    return epoch_loss, epoch_acc

In [ ]:
def train_one_epoch(model, dataloader, optimizer, criterion, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for batch in dataloader:
        
        images = batch['image'].to("cuda:0")
        label = batch['label'].to("cuda:0")

        # Forward
        output = model(images)
        loss = criterion(output, label)

        # Backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Statistics
        running_loss += loss.item() * images.size(0)
        _, preds = output.max(1)
        correct += preds.eq(label).sum().item()
        total += label.size(0)
    epoch_loss = running_loss / total
    epoch_acc = correct / total
    return epoch_loss, epoch_acc

@torch.no_grad()
def validate(model, dataloader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0

    for batch in dataloader:
        images = batch['image'].to("cuda:0")
        label = batch['label'].to("cuda:0")

        # Forward
        output = model(images)
        loss = criterion(output, label)
        running_loss += loss.item() * images.size(0)
        _, preds = output.max(1)
        correct += preds.eq(label).sum().item()
        total += label.size(0)
    epoch_loss = running_loss / total
    epoch_acc = correct / total
    return epoch_loss, epoch_acc

In [ ]:
best_val_acc = 0.0
num_epochs = 40
for epoch in range(num_epochs):
    train_loss, train_acc = train_one_epoch(
        model, train_loader, optimizer, criterion, device
    )

    val_loss, val_acc = validate(
        model, val_loader, criterion, device
    )

    # Save best model
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), "best_model.pth")

    print(
        f"Epoch [{epoch+1}/{num_epochs}] "
        f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f} | "
        f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}"
    )


# ResNet Architectures

In [ ]:
# class BasicBlock(nn.Module):
#     expansion = 1
#     def __init__(self, in_channels, out_channels, stride=1, downsample=None):
#         super().__init__()
#         self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
#         self.bn1 = nn.BatchNorm2d(out_channels)
#         self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
#         self.bn2 = nn.BatchNorm2d(out_channels)
#         self.relu = nn.ReLU(inplace=True)

#         self.downsample = downsample
#         self.strdide = stride

#     def forward(self, x):
#         identity = x
#         out = self.relu(self.bn1(self.conv1(x)))
#         out = self.bn2(self.conv2(out))
#         if self.downsample is not None:
#             identity = self.downsample(x)
#         out += identity
#         out = self.relu(out)
#         return out

In [ ]:
# class ResNet34(nn.Module):
#     def __init__(self, num_classes=1000):
#         super().__init__()
#         self.in_channels = 64

#         # Initial layers
#         self.conv1 = nn.Conv2d(3,64, kernel_size=7, stride=2, padding=3, bias=False)
#         self.bn1 = nn.BatchNorm2d(64)
#         self.relu = nn.ReLU(inplace=True)
#         self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

#         # Stage 1: 3 Blocks, 64 channels
#         self.layer1 = self._make_layer(64,3, stride=1)
#         # Stage 2: 4 blocks, 128 channels
#         self.layer2 = self._make_layer(128, 4, stride=2)
#         # Stage 3: 6 blocks, 256 channels
#         self.layer3 = self._make_layer(256, 6, stride=2)
#         # Stage 4: 3 blocks, 512 channels
#         self.layer4 = self._make_layer(512, 3, stride=2)

#         self.avgpool = nn.AdaptiveAvgPool2d((1,1))
#         self.fc = nn.Linear(512 * BasicBlock.expansion, num_classes)

#     def _make_layer(self, out_channels, blocks, stride):
#         downsample = None

#         if stride != 1 or self.in_channels != out_channels * BasicBlock.expansion:
#             downsample = nn.Sequential(nn.Conv2d(self.in_channels, out_channels * BasicBlock.expansion, kernel_size=1, stride=stride, bias=False),
#                                       nn.BatchNorm2d(out_channels*BasicBlock.expansion)
#                                       )
#         layers = []
#         layers.append(BasicBlock(self.in_channels, out_channels, stride, downsample))

#         self.in_channels = out_channels * BasicBlock.expansion
#         for _ in range(1, blocks):
#             layers.append(BasicBlock(self.in_channels, out_channels))
#         return nn.Sequential(*layers)

#     def forward(self, x):
#         x = self.relu(self.bn1(self.conv1(x)))
#         x = self.maxpool(x)
#         x = self.layer1(x)
#         x = self.layer2(x)
#         x = self.layer3(x)
#         x = self.layer4(x)
        
#         x = self.avgpool(x)
#         x = torch.flatten(x, 1)
#         x = self.fc(x)
        
#         return x

            
            

In [ ]:
# class Bottleneck(nn.Module):
#     expansion = 4

#     def __init__(self, in_channels, out_channels, stride=1, downsample=None):
#         super().__init__()

#         self.conv1 = nn.Conv2d(
#             in_channels, out_channels,
#             kernel_size=1, stride=1, padding=0, bias=False
#         )
#         self.bn1 = nn.BatchNorm2d(out_channels)

#         # STRIDE GOES HERE
#         self.conv2 = nn.Conv2d(
#             out_channels, out_channels,
#             kernel_size=3, stride=stride, padding=1, bias=False
#         )
#         self.bn2 = nn.BatchNorm2d(out_channels)

#         self.conv3 = nn.Conv2d(
#             out_channels, out_channels * self.expansion,
#             kernel_size=1, stride=1, padding=0, bias=False
#         )
#         self.bn3 = nn.BatchNorm2d(out_channels * self.expansion)

#         self.relu = nn.ReLU(inplace=True)
#         self.downsample = downsample

#     def forward(self, x):
#         identity = x

#         out = self.relu(self.bn1(self.conv1(x)))
#         out = self.relu(self.bn2(self.conv2(out)))
#         out = self.bn3(self.conv3(out))

#         if self.downsample is not None:
#             identity = self.downsample(x)

#         out += identity
#         out = self.relu(out)
#         return out


In [ ]:
# class ResNet50(nn.Module):
#     def __init__(self, num_classes=1000):
#         super().__init__()
#         self.in_channels = 64

#         # Initial layers
#         self.conv1 = nn.Conv2d(3,64, kernel_size=7, stride=2, padding=3, bias=False)
#         self.bn1 = nn.BatchNorm2d(64)
#         self.relu = nn.ReLU(inplace=True)
#         self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

#         # Stage 1: 3 Blocks, 64 channels
#         self.layer1 = self._make_layer(64,3, stride=1)
#         # Stage 2: 4 blocks, 128 channels
#         self.layer2 = self._make_layer(128, 4, stride=2)
#         # Stage 3: 6 blocks, 256 channels
#         self.layer3 = self._make_layer(256, 6, stride=2)
#         # Stage 4: 3 blocks, 512 channels
#         self.layer4 = self._make_layer(512, 3, stride=2)

#         self.avgpool = nn.AdaptiveAvgPool2d((1,1))
#         self.fc = nn.Linear(512 * Bottleneck.expansion, num_classes)

#     def _make_layer(self, out_channels, blocks, stride):
#         downsample = None

#         if stride != 1 or self.in_channels != out_channels * Bottleneck.expansion:
#             downsample = nn.Sequential(nn.Conv2d(self.in_channels, out_channels * Bottleneck.expansion, kernel_size=1, stride=stride, bias=False),
#                                       nn.BatchNorm2d(out_channels*Bottleneck.expansion)
#                                       )
#         layers = []
#         layers.append(Bottleneck(self.in_channels, out_channels, stride, downsample))

#         self.in_channels = out_channels * Bottleneck.expansion
#         for _ in range(1, blocks):
#             layers.append(Bottleneck(self.in_channels, out_channels))
#         return nn.Sequential(*layers)

#     def forward(self, x):
#         x = self.relu(self.bn1(self.conv1(x)))
#         x = self.maxpool(x)
#         x = self.layer1(x)
#         x = self.layer2(x)
#         x = self.layer3(x)
#         x = self.layer4(x)
        
#         x = self.avgpool(x)
#         x = torch.flatten(x, 1)
#         x = self.fc(x)
        
#         return x

            
            

In [ ]:
# transform = transforms.Compose([
#     transforms.Resize(256),
#     transforms.RandomCrop(224),
#     transforms.RandomHorizontalFlip(),
#     transforms.ToTensor(),
#     # AlexNetPCAJitter(IMAGENET_EIGVAL, IMAGENET_EIGVEC),
#     transforms.Normalize(
#         mean=[0.485, 0.456, 0.406],
#         std=[0.229, 0.224, 0.225]
#     )
# ])
# test_transform = transforms.Compose([
#     transforms.Resize(256),
#     transforms.CenterCrop(224),
#     transforms.ToTensor(),
#     transforms.Normalize(
#         mean=[0.485, 0.456, 0.406],
#         std=[0.229, 0.224, 0.225]
#     )
# ])

# def preprocess(example, transform):
#     example["image"] = [transform(img) for img in example["image"]]
#     return example

# train_dataset = train_dataset.with_transform(lambda x: preprocess(x, transform))
# valid_dataset = valid_dataset.with_transform(lambda x: preprocess(x, test_transform))
# test_dataset = test_dataset.with_transform(lambda x: preprocess(x, test_transform))

In [ ]:
# # del model, optimizer
# # torch.cuda.empty_cache()
# model_resnet50 = ResNet50(num_classes=200)
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model_resnet50.to(device)
# criterion = nn.CrossEntropyLoss()
# optimizer = optim.SGD(model_resnet50.parameters(),lr=0.01, momentum=0.9, weight_decay=0.5e-4)

In [ ]:
# train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=4)
# val_loader   = DataLoader(valid_dataset, batch_size=64, shuffle=False, num_workers=4)
# test_loader   = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=4)

In [ ]:
# best_val_acc = 0.0
# num_epochs = 40
# for epoch in range(num_epochs):
#     train_loss, train_acc = train_one_epoch(
#         model_resnet50, train_loader, optimizer, criterion, device
#     )

#     val_loss, val_acc = validate(
#         model_resnet50, val_loader, criterion, device
#     )

#     # Save best model
#     if val_acc > best_val_acc:
#         best_val_acc = val_acc
#         torch.save(model_resnet50.state_dict(), "best_model_resnet34.pth")

#     print(
#         f"Epoch [{epoch+1}/{num_epochs}] "
#         f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f} | "
#         f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}"
#     )


In [ ]:
_, test_acc = validate( model, test_loader, criterion, device)
print(test_acc)